### Universal dependencies with __[Stanza](https://stanfordnlp.github.io/stanza/#getting-started)__ and __[spacy_conll](https://spacy.io/universe/project/spacy-conll)__

In [1]:
import pandas as pd
import os
import re
import razdel
import stanza
import six
from spacy_conll import init_parser

In [2]:
news_filepath = '/Users/oksana/Dev/data/april.csv'
news = pd.read_csv(news_filepath)
#                    , dtype={'hash':'uint64'} )
news['hash'] = pd.util.hash_pandas_object(news.link)

news['title'] = news.title.str.replace(r'\n', ' ')
news['all_text'] = news.title.str.cat(news.text, sep='\n', na_rep = '')
news['all_text'] = news.all_text.str.strip()

news['all_text'] = news['all_text'].str.replace('не очень', 'неочень')
news['all_text'] = news['all_text'].str.replace('не дуже', 'недуже')

news.all_text = news.all_text.str.replace(r'^Редактор Цензор\.НЕТ\n', '', flags=re.M)
news.all_text = news.all_text.str.replace(r'Цензор\.НЕТ', 'Цензор')

/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
/Users/oksana/Dev/TextClassification/venv/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [3]:
stanza.download('uk')
stanza.download('ru', package='gsd', processors='tokenize,pos,lemma,depparse')

nlp = stanza.Pipeline('uk', processors='tokenize,lemma')

nlp_uk = init_parser(
        "stanza",
        "uk", 
        is_tokenized = True,
        include_headers=False,
        parser_opts = {'processors': 'tokenize,pos,lemma,depparse'}
    )

nlp_ru = init_parser(
        "stanza",
        "ru", 
        is_tokenized = True,
        include_headers=False,
        parser_opts = {'package': 'gsd', 'processors': 'tokenize,pos,lemma,depparse'}
    )

2021-06-07 20:34:09 INFO: Downloading default packages for language: uk (Ukrainian)...
2021-06-07 20:34:10 INFO: File exists: /Users/oksana/stanza_resources/uk/default.zip.
2021-06-07 20:34:15 INFO: Finished downloading models and saved to /Users/oksana/stanza_resources.
2021-06-07 20:34:15 INFO: Downloading these customized packages for language: ru (Russian)...
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| pretrain  | gsd     |

2021-06-07 20:34:15 INFO: File exists: /Users/oksana/stanza_resources/ru/tokenize/gsd.pt.
2021-06-07 20:34:15 INFO: File exists: /Users/oksana/stanza_resources/ru/pos/gsd.pt.
2021-06-07 20:34:15 INFO: File exists: /Users/oksana/stanza_resources/ru/lemma/gsd.pt.
2021-06-07 20:34:16 INFO: File exists: /Users/oksana/stanza_resources/ru/depparse/gsd.pt.
2021-06-07 20:34:17 INFO: File exists: /Users/oksana/stanza_resources/ru/pretrain/gsd.pt.
2021-06-07 20:34:17 INF

In [24]:
s1="Все прошло плохо , зато мы не хотим после этого расстраиваться"
doc = nlp_ru(s1)
conll = doc._.conll_str
print(conll)

1	Все	весь	PRON	DT	_	2	nsubj	_	_
2	прошло	пройти	VERB	VBC	_	0	root	_	_
3	плохо	плохо	ADV	RB	_	2	advmod	_	_
4	,	,	PUNCT	,	_	8	punct	_	_
5	зато	зато	SCONJ	IN	_	8	mark	_	_
6	мы	мы	PRON	PRP	_	8	nsubj	_	_
7	не	не	PART	NEG	_	8	advmod	_	_
8	хотим	хотеть	VERB	VBC	_	2	advcl	_	_
9	после	после	ADP	IN	_	10	case	_	_
10	этого	это	PRON	DT	_	8	obl	_	_
11	расстраиваться	расстраиваться	VERB	VB	_	8	xcomp	_	SpaceAfter=No



In [4]:
import re
import six
import razdel


ACCENT = six.unichr(769)
WORD_TOKENIZATION_RULES = re.compile(r"""
[\w""" + ACCENT + """]+://(?:[a-zA-Z]|[0-9]|[$-_@.&+])+
|[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+
|[0-9]+-[а-яА-ЯіїІЇєЄґҐ'’`""" + ACCENT + """]+
|[+-]?[0-9](?:[0-9,.-]*[0-9])?
|[\w""" + ACCENT + """](?:[\w'’`-""" + ACCENT + """]?[\w""" + ACCENT + """]+)*
|[\w""" + ACCENT + """].(?:\[\w""" + ACCENT + """].)+[\w""" + ACCENT + """]?
|["#$%&*+,/:;<=>@^`~…\\(\\)⟨⟩{}\[\|\]‒–—―«»“”‘’'№]
|[.!?]+
|-+
""", re.X | re.U)


ABBRS = """
ім.
в.
о.
т.
п.
д.
под.
ін.
вул.
просп.
бул.
пров.
пл.
г.
р.
див.
п.
с.
м.
н.
е.
адмін.
к.
геогр.
обл.
смт.
авт.
адм.
акад.
англ.
арк.
арт.
археол.
арх.
архіт.
асист.
асоц.
б.
буд.
бух.
бюдж.
вет.
вид.
викл.
відкр.
дип.
діагр.
екол.
екон.
євр.
журн.
зобр.
іл.
інв.
інд.
інж.
іст.
каф.
кл.
коеф.
лаб.
лінгв.
літ.
мат.
мед.
мех.
міс.
муз.
нар.
нац.
орг.
офіц.
пед.
пр.
проф.
публ.
рис.
мал.
pp.
рос.
св.
сл.
ст.
студ.
табл.
тис.
укр.
упр.
фіз.
фін.
ц.
""".strip().split()


def tokenize_sents(string):
    string = six.text_type(string)
    spans = []
    for match in re.finditer('[^\s]+', string):
        spans.append(match)
    spans_count = len(spans)

    rez = []
    off = 0

    for i in range(spans_count):
        tok = string[spans[i].start():spans[i].end()]
        if i == spans_count - 1:
            rez.append(string[off:spans[i].end()])
        elif tok[-1] in ['.', '!', '?', '…', '»', "'", "\""]:
            # tok1 = tok[re.search('[.!?…»]', tok).start() - 1]
            next_tok = string[spans[i + 1].start():spans[i + 1].end()]
            if (next_tok[0].isupper() or next_tok[0] in ["'", "\"", "«"]) \
                    and not ((len(tok) == 2 and tok[0].isupper()) \
                             or tok[0] == '('
                             or tok in ABBRS):
                rez.append(string[off:spans[i].end()])
                off = spans[i + 1].start()

    return rez


def text_to_sent(text, lang):
    rez = []
    if lang == 'uk':
        for part in text.split('\n'):
            rez += tokenize_sents(part)
    elif lang=='ru':
        for part in text.split('\n'):
            rez += [s.text for s in razdel.sentenize(part)]
    return rez


def sent_to_words(text, lang):
    if lang == 'uk':
        return re.findall(WORD_TOKENIZATION_RULES, text)
    elif lang == 'ru':
        return [tkn.text for tkn in razdel.tokenize(text)]
    return None

# def tokenize(text, lang):
#     res = []
#     for sent in text_to_sent(text, lang):
#         tokens = []
#         for word in sent_to_words(sent, lang):
#             tokens.append(word)
#         res.append(tokens)
#     return res

def tokenize(text, lang):
    res = []
    for sent in text_to_sent(text, lang):
        tokens = []
        for word in sent_to_words(sent, lang):
            tokens.append(word)
        res.append(' '.join(tokens))
    return '\n'.join(res)

In [5]:
news.columns

Index(['title', 'text', 'subtitle', 'link', 'domain', 'datetime', 'views',
       'created_at', 'category', 'language', 'id', 'domain_alias',
       'mycategory', 'mentions', 'hash', 'all_text'],
      dtype='object')

In [6]:
%%time
news['tokenized'] = news.apply(lambda row: tokenize(row.all_text, row.language), axis=1)

CPU times: user 4min 14s, sys: 2.7 s, total: 4min 16s
Wall time: 4min 24s


In [7]:
def save_to_conllu(doc, link_hash, out_dir = '/Users/oksana/Dev/Politicians_in_media/april_conllu/conllu_uk/'):
    out_file = out_dir + "{}.conll".format(link_hash)
    for sent_idx, sent in enumerate(doc.sents, 1):
        header = ['### ', sent_idx, link_hash]
        pd.DataFrame([header]).to_csv(out_file, sep='\t', index=False, header=None, mode='a')
        sent._.conll_pd.to_csv(out_file, index=False, sep="\t", encoding='utf-8', mode='a', header=None)
        with open(out_file, 'a') as f:
            f.write('\n')

In [8]:
news = news.drop(columns=['all_text', 'text'])

In [9]:
news_uk = news[news.language=='uk']
news_ru = news[news.language=='ru']

In [15]:
news_uk.columns

Index(['id', 'link', 'title', 'domain', 'datetime', 'views', 'category',
       'language', 'domain_alias', 'found_names_str', 'mycategory', 'mentions',
       'tone_words', 'hash', 'tokenized'],
      dtype='object')

In [10]:
def process_news(news, nlp=nlp_uk, start=0, finish=1000, step=100, out_dir='/Users/oksana/Dev/Politicians_in_media/april_conllu/conllu_uk/'):
    for k in range(start, finish, step):
        try:
            del news_part
        except:
            pass    
        news_part = news.iloc[k:k + step].copy()
        news_part = news_part[news_part.mentions.notna()]
        news_part['docs'] = news_part.tokenized.apply(nlp)
        news_part.apply(lambda row: save_to_conllu(row.docs, row.hash, out_dir), axis=1)

In [11]:
news[['link', 'hash', 'id']].to_csv('link_hash_id_april.csv')

In [14]:
news_uk.shape

(123238, 15)

In [18]:
%%time
process_news(news_uk, nlp_uk, start=100000, finish=len(news_uk), step=200)

CPU times: user 1h 58min 13s, sys: 15min 45s, total: 2h 13min 58s
Wall time: 2h 13min 27s


In [17]:
%%time
process_news(news_uk, nlp_uk, start=50000, finish=100000, step=200)

CPU times: user 4h 38min 3s, sys: 35min 5s, total: 5h 13min 9s
Wall time: 4h 58min 2s


In [14]:
news_part['docs'] = news_part.tokenized.apply(nlp_uk)
news_part.apply(lambda row: save_to_conllu(row.docs, row.hash, '/Users/oksana/Dev/Politicians_in_media/march_conllu/ukr/'), axis=1)

25752    None
25753    None
25756    None
25758    None
25765    None
         ... 
26737    None
26739    None
26740    None
26749    None
26753    None
Length: 217, dtype: object

In [41]:
%%time
process_news(news_uk, nlp_uk, start=80000, finish=85000, step=200)

CPU times: user 25min 1s, sys: 3min 29s, total: 28min 30s
Wall time: 27min 30s


In [33]:
n = os.listdir('/Users/oksana/Dev/Politicians_in_media/march_conllu/ukr/')
len(n)

38000

### Sentiment analysis with __[UUUSA](https://github.com/aghie/uuusa)__   ( __[article](https://arxiv.org/abs/1606.05545)__,  __[manual](http://grupolys.org/software/UUUSA/uuusa-user-manual.pdf)__ )

In [ ]:
java -Dfile.encoding=UTF-8 -jar -Xmx2g samulan-0.1.0.jar \ 
-s UkSentiData \
-r configuration_uk.xml \
-c parsed.conll \
-p samulan.properties \
-v true
-o output.txt

In [ ]:
for file in *.conll; do java -Dfile.encoding=UTF-8 -jar -Xmx2g ../samulan-0.1.1.jar \
-s ../UkSentiData \
-r ../configuration_uk.xml \
-c $file \
-p ../samulan.properties\
-sc so \
-o ../output/$file; done

    1.0		Каменських назвала свій головний привід для радості , і це не Потап : """" Згодні ? """"	 [The analysis took: 0.060065182 seg.] [Accumulated time is: 0.060065182]
    3.0		Популярна українська співачка Настя Каменських , яка минулого року вийшла заміж за свого колегу Олексія Потапенка , часто радує публіку яскравими постами в своєму Instagram .	 [The analysis took: 0.001861094 seg.] [Accumulated time is: 0.061926276]
    1.0		На карантині у зірки з'явилося більше часу для спілкування з фанатами , чим вона активно користується .	 [The analysis took: 0.001455214 seg.] [Accumulated time is: 0.06338149]
    0.0		Читайте Знай в Google News !	 [The analysis took: 7.0612E-4 seg.] [Accumulated time is: 0.06408761
    2.0		Нещодавно дружина Потапа розмістила свіже фото , на якому позує біля пальми в легкому жіночному образі – жовтих шортах , білій сорочці та верху від купальника теж білого кольору .	 [The analysis took: 0.002728221 seg.] [Accumulated time is: 0.066815831]
    1.0		У підписі Каменських зазначила , що п'ятниця вже є приводом для радості .	 [The analysis took: 0.002728221 seg.] [Accumulated time is: 0.066815831]


In [ ]:
sentiment_files = os.listdir(sentiment_dir)
res = []
for file in sentiment_files:
    s = pd.read_csv(file, sep='\t', usecols=[0], header=None)[0]
    res.append((file.strip('.conll'), s.sum(), s.astype(str).str.cat(sep=';')))
df = pd.DataFrame(res, columns=['hash', 'sentiment', 'sent_list'])
df.hash = df.hash.astype('uint64')